<a href="https://colab.research.google.com/github/YonggunJung/Project2-Review/blob/main/4_%EC%A0%9C%EB%A1%9C%EC%83%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os

import torch
import torch.nn as nn
from tqdm import trange
from transformers import ElectraForSequenceClassification, ElectraModel, AdamW
from transformers import ElectraTokenizer # AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
from datasets import load_metric
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import copy

from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

# 쿠다 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [ ]:
print(torch.__version__)
# GPU 사용
device = torch.device('cuda')
print(device)
print(torch.cuda.is_available())

1.12.1+cu113
cuda
True


In [ ]:
df = pd.read_csv('./data/긍부중모델용.csv', encoding='cp949')
df

,review
0,여러가지 맛을 한번에 맛 볼 수 있어서 좋아요
1,정말 맛있네요 ㅠㅠ 맛 여러가지 섞인 땡큐로 샀어요
2,맛나죠 말이필요없는정도ㅎ
3,5시 새벽배송받아 힘들어하는 수험생 아들에게 화이팅^^ 메모와 함께 책상에 살짝 올...
4,초코튜브 갯수도 꽤 되고 맛도 좋습니다. 세종류 다 맛있고 저는 딸기맛이 좋네요
5,세일할때 쟁여두는 상품입니다
6,당 떨어질 때 하나씩 먹기 좋아요
7,같이 일하는분 간식으로 드렷더니 좋아하시네요
8,초콜릿 새콤달콤 맛있어요
9,가끔 먹어요 맛있어서~


In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator").to(device)


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

In [ ]:
model.load_state_dict(torch.load("./model/model.pt", map_location=device))

RuntimeError: PytorchStreamReader failed locating file data/0: file not found